In [477]:
# Подключение к Google drive

#from google.colab import drive
#drive.mount('/content/drive')

In [478]:
# Загрузка библиотек

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Датасет: freMPL (French Motor Personal Line datasets)

Источник данных: http://cas.uqam.ca/

Продукт: КАСКО

Набор из 10 датасетов частного французского автостраховщика. Каждый датасет содержит характеристики риска, суммы величин страховых требований и истории страховых исков по около 30 000 страховых полисов за 2004 год.

In [479]:
# Загрузка набора данных в pandas DataFrame

df = pd.read_csv('../input/difffer/freMPL-R.csv', low_memory=False)

In [480]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343080 entries, 0 to 343079
Data columns (total 31 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Exposure           343080 non-null  float64
 1   LicAge             343080 non-null  int64  
 2   RecordBeg          343080 non-null  object 
 3   RecordEnd          181115 non-null  object 
 4   VehAge             177880 non-null  object 
 5   Gender             343080 non-null  object 
 6   MariStat           343080 non-null  object 
 7   SocioCateg         343080 non-null  object 
 8   VehUsage           343080 non-null  object 
 9   DrivAge            343080 non-null  int64  
 10  HasKmLimit         343080 non-null  int64  
 11  BonusMalus         343080 non-null  int64  
 12  VehBody            145780 non-null  object 
 13  VehPrice           145780 non-null  object 
 14  VehEngine          145780 non-null  object 
 15  VehEnergy          145780 non-null  object 
 16  Ve

In [481]:
df.head()

,Exposure,LicAge,RecordBeg,RecordEnd,VehAge,Gender,MariStat,SocioCateg,VehUsage,DrivAge,...,ClaimInd,Dataset,DeducType,ClaimNbResp,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,OutUseNb,RiskArea
0,0.583,366,2004-06-01,NaN,2,Female,Other,CSP1,Professional,55,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.200,187,2004-10-19,NaN,0,Male,Alone,CSP55,Private+trip to office,34,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.083,169,2004-07-16,2004-08-16,1,Female,Other,CSP1,Professional,33,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.375,170,2004-08-16,NaN,1,Female,Other,CSP1,Professional,34,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.500,224,2004-01-01,2004-07-01,3,Male,Other,CSP47,Professional,53,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##Проверка качества заполнения данных

Поскольку freMPL состоит из 10 отдельных датасетов, каждый из которых имеет различные переменные, есть смысл заранее посмотреть, какие датасеты стоит использовать, чтобы затем не возникало проблем с большим количеством пропущенных данных.

In [482]:
df.Dataset

0          1
1          1
2          1
3          1
4          1
          ..
343075    10
343076    10
343077    10
343078    10
343079    10
Name: Dataset, Length: 343080, dtype: int64

In [483]:
# Смотрим, какие факторы пропущены в каждом из датасетов

dct = {}
for i in range(1,11):
    print(df.Dataset == i)
    _x = df.loc[df.Dataset == i].notnull().sum()
    dct[i] = list(_x[_x == 0].index)

# print('Dataset  Missing Variables')
# for x in range(1,11):
#     print(x,'\t',dct[x])

dct[i]

0          True
1          True
2          True
3          True
4          True
          ...  
343075    False
343076    False
343077    False
343078    False
343079    False
Name: Dataset, Length: 343080, dtype: bool
0         False
1         False
2         False
3         False
4         False
          ...  
343075    False
343076    False
343077    False
343078    False
343079    False
Name: Dataset, Length: 343080, dtype: bool
0         False
1         False
2         False
3         False
4         False
          ...  
343075    False
343076    False
343077    False
343078    False
343079    False
Name: Dataset, Length: 343080, dtype: bool
0         False
1         False
2         False
3         False
4         False
          ...  
343075    False
343076    False
343077    False
343078    False
343079    False
Name: Dataset, Length: 343080, dtype: bool
0         False
1         False
2         False
3         False
4         False
          ...  
343075    False
343076    Fa

['VehBody',
 'VehPrice',
 'VehEngine',
 'VehEnergy',
 'VehMaxSpeed',
 'VehClass',
 'RiskVar',
 'Garage',
 'DeducType']

Наборы данных 1-4 не содержат информации о количестве страховых требований, а 5-10 не содержат информации по характеристикам транспортного средства. Тем не менее, наборы 5-9 имеют одинаковые пропущенные факторы, имеет смысл попробовать их объединить.

In [484]:
# Объединяем наборы данных 5-9, удаляем пустые столбцы, удаляем дубликаты

df59 = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df59 = df59.drop(['Dataset'], axis=1)
df59 = df59.dropna(axis=1, how='all')
print('With duplicates\t\t', len(df59))
df59 = df59.drop_duplicates()
print('Without duplicates\t', len(df59))

With duplicates		 165200
Without duplicates	 115155


Внутри каждого набора данных имеется около 10 000 наблюдений дубликатов. Все дубликаты были удалены.

In [485]:
df59.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115155 entries, 145780 to 310979
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Exposure           115155 non-null  float64
 1   LicAge             115155 non-null  int64  
 2   RecordBeg          115155 non-null  object 
 3   RecordEnd          59455 non-null   object 
 4   Gender             115155 non-null  object 
 5   MariStat           115155 non-null  object 
 6   SocioCateg         115155 non-null  object 
 7   VehUsage           115155 non-null  object 
 8   DrivAge            115155 non-null  int64  
 9   HasKmLimit         115155 non-null  int64  
 10  BonusMalus         115155 non-null  int64  
 11  ClaimAmount        115155 non-null  float64
 12  ClaimInd           115155 non-null  int64  
 13  ClaimNbResp        115155 non-null  float64
 14  ClaimNbNonResp     115155 non-null  float64
 15  ClaimNbParking     115155 non-null  float64
 1

In [486]:
df = df59

In [487]:
# Вспомогательный столбец для суммирования числа полисов

df['PolicyCount'] = 1

In [488]:
# Вспомогательный столбец для суммирования числа полисов с убытками

df['ClaimCount'] = df['ClaimAmount'] > 0

In [489]:
df['ClaimCount'] == 12

145780    False
145781    False
145782    False
145783    False
145784    False
          ...  
310975    False
310976    False
310977    False
310978    False
310979    False
Name: ClaimCount, Length: 115155, dtype: bool

In [490]:
# Вспомогательный столбец для суммирования числа полисов без убытков

df['NoClaimCount'] = df.PolicyCount - df.ClaimCount

In [491]:
df[['PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].head()

,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount
145780,1,0.0,False,1
145781,1,0.0,False,1
145782,1,0.0,False,1
145783,1,0.0,False,1
145784,1,0.0,False,1


##Основные зависимости

Смотрим зависимость от возраста

In [492]:
df_group_age = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(df['DrivAge']).sum()
df_group_age = df_group_age.reset_index()

In [493]:
df_group_age.head()

,DrivAge,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount
0,20,81.637,189,148885.858006,28,161
1,21,164.026,402,85952.677341,55,347
2,22,290.233,687,241769.777946,98,589
3,23,371.904,913,179928.090634,114,799
4,24,438.098,1094,272854.509970,128,966


In [494]:
fig = px.bar(df_group_age, x='DrivAge', y='PolicyCount', title='Распределение числа полисов по возрасту')
fig.show()

**Вопрос:** Чем можно объяснить основные закономерности в распределении по возрасту?

In [495]:
fig = go.Figure(data=[go.Bar(name='No Claims', x=df_group_age.DrivAge, y=df_group_age.NoClaimCount),
                      go.Bar(name='With Claims', x=df_group_age.DrivAge, y=df_group_age.ClaimCount)])
fig.update_layout(barmode='stack', xaxis_title='DrivAge', yaxis_title='PolicyCount',title='Распределение числа полисов по возрасту с убытками и без')
fig.show()

In [496]:
df_group_age['Freq'] = df_group_age.ClaimCount / df_group_age.Exposure
df_group_age['AvgClaim'] = df_group_age.ClaimAmount / df_group_age.ClaimCount

In [497]:
df_group_age.head()

,DrivAge,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim
0,20,81.637,189,148885.858006,28,161,0.342982,5317.352072
1,21,164.026,402,85952.677341,55,347,0.335313,1562.775952
2,22,290.233,687,241769.777946,98,589,0.337660,2467.038550
3,23,371.904,913,179928.090634,114,799,0.306531,1578.316585
4,24,438.098,1094,272854.509970,128,966,0.292172,2131.675859


In [498]:
fig = px.bar(df_group_age, x='DrivAge', y='Freq', title='Зависимость частоты убытков от возраста')
fig.show()

In [499]:
fig = px.bar(df_group_age, x='DrivAge', y='AvgClaim', title='Зависимость среднего убытка от возраста')
fig.show()

Смотрим зависимость от стажа

In [500]:
df_group_LicAge = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(df['LicAge']).sum()
df_group_LicAge = df_group_LicAge.reset_index()
df_group_LicAge['Freq'] = df_group_LicAge.ClaimCount / df_group_LicAge.Exposure
df_group_LicAge['AvgClaim'] = df_group_LicAge.ClaimAmount / df_group_LicAge.ClaimCount

In [501]:
df_group_LicAge.head()

,LicAge,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim
0,2,0.341,1,180.955287,1,0,2.932551,180.955287
1,6,0.147,1,0.000000,0,1,0.000000,NaN
2,24,57.532,128,126155.939275,25,103,0.434541,5046.237571
3,25,51.886,111,37246.590634,20,91,0.385460,1862.329532
4,26,36.441,75,35945.227795,12,63,0.329299,2995.435650


In [502]:
fig = px.bar(df_group_LicAge, x='LicAge', y='Freq', title='Зависимость частоты убытков от стажа')
fig.show()

In [503]:
fig = px.bar(df_group_LicAge, x='LicAge', y='AvgClaim', title='Зависимость среднего убытка от стажа')
fig.show()

Смотрим зависимость от пола

In [504]:
df_group_Gender = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(df['Gender']).sum()
df_group_Gender = df_group_Gender.reset_index()
df_group_Gender['Freq'] = df_group_Gender.ClaimCount / df_group_Gender.Exposure
df_group_Gender['AvgClaim'] = df_group_Gender.ClaimAmount / df_group_Gender.ClaimCount

In [505]:
df_group_Gender

,Gender,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim
0,Female,19073.537,43339,8.057192e+06,4087,39252,0.214276,1971.419679
1,Male,31808.614,71816,1.399185e+07,6782,65034,0.213213,2063.086503


In [506]:
fig = px.bar(df_group_Gender, x='Gender', y='Freq', title='Зависимость частоты убытков от пола')
fig.show()

In [507]:
fig = px.bar(df_group_Gender, x='Gender', y='AvgClaim', title='Зависимость среднего убытка от пола')
fig.show()

**Вопрос:** Можно ли считать пол значимым фактором для данной выборки?

Смотрим зависимость от пола и возраста

In [508]:
df_group_Gender_Age = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount', 'Gender', 'DrivAge']].groupby(['Gender', 'DrivAge']).sum()
df_group_Gender_Age = df_group_Gender_Age.reset_index()
df_group_Gender_Age['Freq'] = df_group_Gender_Age.ClaimCount / df_group_Gender_Age.Exposure
df_group_Gender_Age['AvgClaim'] = df_group_Gender_Age.ClaimAmount / df_group_Gender_Age.ClaimCount

In [509]:
df_group_Gender_Age.head()

,Gender,DrivAge,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim
0,Female,20,54.222,116,65625.029003,19,97,0.350411,3453.948895
1,Female,21,108.516,261,60939.667976,34,227,0.313318,1792.343176
2,Female,22,179.790,419,125370.990634,53,366,0.294788,2365.490389
3,Female,23,224.401,547,96561.737764,70,477,0.311942,1379.453397
4,Female,24,239.689,609,116515.528097,64,545,0.267013,1820.555127


In [510]:
fig = px.line(df_group_Gender_Age, x='DrivAge', y='Freq', color='Gender', title='Зависимость частоты убытков от пола и возраста')
fig.show()

In [511]:
fig = px.line(df_group_Gender_Age, x='DrivAge', y='AvgClaim', color='Gender', title='Зависимость среднего убытка от пола и возраста')
fig.show()

Смотрим зависимость от семейного статуса

In [512]:
df_group_MariStat = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(df['MariStat']).sum()
df_group_MariStat = df_group_MariStat.reset_index()
df_group_MariStat['Freq'] = df_group_MariStat.ClaimCount / df_group_MariStat.Exposure
df_group_MariStat['AvgClaim'] = df_group_MariStat.ClaimAmount / df_group_MariStat.ClaimCount

In [513]:
df_group_MariStat

,MariStat,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim
0,Alone,7676.141,17714,4.503143e+06,1769,15945,0.230454,2545.586579
1,Other,43206.010,97441,1.754590e+07,9100,88341,0.210619,1928.121124


In [514]:
fig = px.bar(df_group_MariStat, x='MariStat', y='Freq', title='Зависимость частоты убытков от семейного статуса')
fig.show()

In [515]:
fig = px.bar(df_group_MariStat, x='MariStat', y='AvgClaim', title='Зависимость среднего убытка от семейного статуса')
fig.show()

**Вопрос:** Соответствует ли полученная зависимость от семейного статуса бытовой логике?

Смотрим зависимость от пола, семейного статуса и возраста

In [516]:
df_group_Gender_MariStat_Age = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount', 'Gender', 'DrivAge', 'MariStat']].groupby(['Gender', 'DrivAge', 'MariStat']).sum()
df_group_Gender_MariStat_Age = df_group_Gender_MariStat_Age.reset_index()
df_group_Gender_MariStat_Age['Freq'] = df_group_Gender_MariStat_Age.ClaimCount / df_group_Gender_MariStat_Age.Exposure
df_group_Gender_MariStat_Age['AvgClaim'] = df_group_Gender_MariStat_Age.ClaimAmount / df_group_Gender_MariStat_Age.ClaimCount
df_group_Gender_MariStat_Age['Gender_MariStat'] = df_group_Gender_MariStat_Age['Gender'] + '_' + df_group_Gender_MariStat_Age['MariStat']

In [517]:
df_group_Gender_MariStat_Age.head()

,Gender,DrivAge,MariStat,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim,Gender_MariStat
0,Female,20,Alone,50.619,107,64206.418731,18,89,0.355598,3567.023263,Female_Alone
1,Female,20,Other,3.603,9,1418.610272,1,8,0.277546,1418.610272,Female_Other
2,Female,21,Alone,95.044,226,53685.503021,30,196,0.315643,1789.516767,Female_Alone
3,Female,21,Other,13.472,35,7254.164955,4,31,0.296912,1813.541239,Female_Other
4,Female,22,Alone,140.695,330,92026.909668,41,289,0.291410,2244.558772,Female_Alone


In [518]:
fig = px.line(df_group_Gender_MariStat_Age, x='DrivAge', y='Freq', color='Gender_MariStat', title='Зависимость частоты убытков от пола, семейного статуса и возраста')
fig.show()

In [519]:
df['DrivAge_5'] = round(df['DrivAge']/5,0)
df_group_Gender_MariStat_Age_5 = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount', 'Gender', 'DrivAge_5', 'MariStat']].groupby(['Gender', 'DrivAge_5', 'MariStat']).sum()
df_group_Gender_MariStat_Age_5 = df_group_Gender_MariStat_Age_5.reset_index()
df_group_Gender_MariStat_Age_5['Freq'] = df_group_Gender_MariStat_Age_5.ClaimCount / df_group_Gender_MariStat_Age_5.Exposure
df_group_Gender_MariStat_Age_5['AvgClaim'] = df_group_Gender_MariStat_Age_5.ClaimAmount / df_group_Gender_MariStat_Age_5.ClaimCount
df_group_Gender_MariStat_Age_5['Gender_MariStat'] = df_group_Gender_MariStat_Age_5['Gender'] + '_' + df_group_Gender_MariStat_Age_5['MariStat']

In [520]:
df_group_Gender_MariStat_Age_5.head()

,Gender,DrivAge_5,MariStat,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim,Gender_MariStat
0,Female,4.0,Alone,286.358,663,209918.831420,89,574,0.310800,2358.638555,Female_Alone
1,Female,4.0,Other,56.170,133,42016.856193,17,116,0.302653,2471.579776,Female_Other
2,Female,5.0,Alone,860.806,2020,371202.553776,223,1797,0.259060,1664.585443,Female_Alone
3,Female,5.0,Other,554.395,1383,276640.738369,149,1234,0.268761,1856.649251,Female_Other
4,Female,6.0,Alone,741.134,1714,276705.695166,164,1550,0.221283,1687.229849,Female_Alone


In [521]:
fig = px.line(df_group_Gender_MariStat_Age_5, x='DrivAge_5', y='Freq', color='Gender_MariStat', title='Зависимость частоты убытков от пола, семейного статуса и возраста')
fig.show()

##Прочие зависимости

Смотрим зависимость от социального статуса

In [522]:
df_group_SocioCateg = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(df['SocioCateg']).sum()
df_group_SocioCateg = df_group_SocioCateg.reset_index()
df_group_SocioCateg['Freq'] = df_group_SocioCateg.ClaimCount / df_group_SocioCateg.Exposure
df_group_SocioCateg['AvgClaim'] = df_group_SocioCateg.ClaimAmount / df_group_SocioCateg.ClaimCount

In [523]:
df_group_SocioCateg = df_group_SocioCateg.sort_values('Freq', ascending=False).reset_index()
df_group_SocioCateg

,index,SocioCateg,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim
0,39,CSP62,0.334,3,8.448837e+03,1,2,2.994012,8448.837462
1,33,CSP56,4.130,26,4.504335e+03,2,24,0.484262,2252.167523
2,11,CSP3,29.738,67,2.496127e+04,13,54,0.437151,1920.097885
3,27,CSP49,97.943,215,9.315982e+04,36,179,0.367561,2587.772734
4,36,CSP6,178.238,385,1.151222e+05,52,333,0.291745,2213.889199
5,25,CSP47,101.373,211,7.615978e+04,28,183,0.276208,2719.992048
6,24,CSP46,1216.246,2582,5.298546e+05,309,2273,0.254060,1714.739694
7,20,CSP42,874.043,1976,3.773718e+05,219,1757,0.250560,1723.159066
8,9,CSP26,590.191,1376,2.463719e+05,147,1229,0.249072,1675.999451
9,26,CSP48,985.892,2157,6.906043e+05,245,1912,0.248506,2818.793038


In [524]:
fig = px.bar(df_group_SocioCateg, x='SocioCateg', y='PolicyCount', title='Распределение числа полисов по социальному статусу')
fig.show()

In [525]:
fig = px.bar(df_group_SocioCateg, x='SocioCateg', y='Freq', title='Зависимость частоты убытков от социального статуса')
fig.show()

In [526]:
fig = px.bar(df_group_SocioCateg, x='SocioCateg', y='AvgClaim', title='Зависимость среднего убытка от социального статуса')
fig.show()

In [527]:
df_group_SocioCategAgg = df.copy()
df_group_SocioCategAgg['SocioCateg'] = df_group_SocioCategAgg['SocioCateg'].str.slice(0,4)
df_group_SocioCategAgg = df_group_SocioCategAgg[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(df_group_SocioCategAgg['SocioCateg']).sum()
df_group_SocioCategAgg = df_group_SocioCategAgg.reset_index()
df_group_SocioCategAgg['Freq'] = df_group_SocioCategAgg.ClaimCount / df_group_SocioCategAgg.Exposure
df_group_SocioCategAgg['AvgClaim'] = df_group_SocioCategAgg.ClaimAmount / df_group_SocioCategAgg.ClaimCount
df_group_SocioCategAgg

,SocioCateg,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim
0,CSP1,1267.854,2740,6.432906e+05,290,2450,0.228733,2218.243495
1,CSP2,1495.440,3254,5.245311e+05,320,2934,0.213984,1639.159543
2,CSP3,538.613,1210,1.625085e+05,111,1099,0.206085,1464.040745
3,CSP4,3491.754,7648,1.853505e+06,889,6759,0.254600,2084.932493
4,CSP5,32697.718,75456,1.534938e+07,7286,68170,0.222829,2106.695084
5,CSP6,11384.500,24833,3.516266e+06,1972,22861,0.173218,1783.096351
6,CSP7,6.272,14,-4.366704e+02,1,13,0.159439,-436.670393


In [528]:
fig = px.bar(df_group_SocioCategAgg, x='SocioCateg', y='PolicyCount', title='Распределение числа полисов по социальному статусу')
fig.show()

In [529]:
fig = px.bar(df_group_SocioCategAgg, x='SocioCateg', y='Freq', title='Зависимость частоты убытков от социального статуса')
fig.show()

In [530]:
fig = px.bar(df_group_SocioCategAgg, x='SocioCateg', y='AvgClaim', title='Зависимость среднего убытка от социального статуса')
fig.show()

# Домашнее задание


<HR>

In [531]:
df.iloc[:10, :]

,Exposure,LicAge,RecordBeg,RecordEnd,Gender,MariStat,SocioCateg,VehUsage,DrivAge,HasKmLimit,...,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,OutUseNb,RiskArea,PolicyCount,ClaimCount,NoClaimCount,DrivAge_5
145780,0.083,332,2004-01-01,2004-02-01,Male,Other,CSP50,Professional,46,0,...,1.0,0.0,0.0,0.0,0.0,9.0,1,False,1,9.0
145781,0.916,333,2004-02-01,NaN,Male,Other,CSP50,Professional,46,0,...,1.0,0.0,0.0,0.0,0.0,9.0,1,False,1,9.0
145782,0.550,173,2004-05-15,2004-12-03,Male,Other,CSP50,Private+trip to office,32,0,...,2.0,0.0,0.0,0.0,0.0,7.0,1,False,1,6.0
145783,0.089,364,2004-11-29,NaN,Female,Other,CSP55,Private+trip to office,52,0,...,0.0,0.0,0.0,0.0,0.0,8.0,1,False,1,10.0
145784,0.233,426,2004-02-07,2004-05-01,Male,Other,CSP60,Private,57,0,...,0.0,0.0,0.0,0.0,0.0,7.0,1,False,1,11.0
145785,0.666,429,2004-05-01,NaN,Male,Other,CSP60,Private,57,0,...,0.0,0.0,0.0,0.0,0.0,7.0,1,False,1,11.0
145786,0.080,461,2004-04-02,2004-05-01,Male,Other,CSP48,Professional,58,0,...,1.0,0.0,0.0,2.0,1.0,5.0,1,False,1,12.0
145787,0.666,462,2004-05-01,NaN,Male,Other,CSP48,Professional,58,0,...,1.0,0.0,0.0,2.0,1.0,5.0,1,False,1,12.0
145788,0.173,405,2004-10-29,NaN,Female,Other,CSP50,Private+trip to office,53,0,...,0.0,0.0,0.0,0.0,0.0,10.0,1,False,1,11.0
145789,0.474,386,2004-01-01,2004-06-22,Male,Other,CSP55,Private+trip to office,57,0,...,0.0,0.0,0.0,1.0,0.0,6.0,1,False,1,11.0


In [532]:
VehUsage = df.groupby(df.VehUsage).sum()
VehUsage = VehUsage.reset_index()

In [533]:
px.bar(VehUsage, x = 'VehUsage', y = 'ClaimCount', title = 'Количество выплат в зависимости от профессии')

In [534]:
VehUsage.head()

,VehUsage,Exposure,LicAge,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,ClaimNbResp,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,OutUseNb,RiskArea,PolicyCount,ClaimCount,NoClaimCount,DrivAge_5
0,Private,17385.293,16848379,2344584,8716,2152725,5.630104e+06,3066,9679.0,9824.0,3304.0,2315.0,11167.0,6467.0,309132.0,38839,3066,35773,468872.0
1,Private+trip to office,25945.001,15313583,2500462,3862,3723842,1.232871e+07,5812,14729.0,19917.0,4192.0,5156.0,23032.0,17187.0,465170.0,59834,5812,54022,499891.0
2,Professional,6574.757,4771108,684973,82,827632,3.658391e+06,1718,4821.0,5635.0,1419.0,1225.0,6842.0,5025.0,114912.0,14302,1718,12584,136862.0
3,Professional run,977.100,731242,104054,0,120389,4.318442e+05,273,768.0,1002.0,226.0,191.0,1279.0,764.0,17452.0,2180,273,1907,20768.0


In [535]:
fig = px.bar(VehUsage, x='VehUsage', y='NoClaimCount',  title='Зависимость безубыточности в зависимости от использования')
fig.show()

In [536]:
fig = go.Figure(data=[go.Bar(name='No Claims', x=VehUsage.VehUsage, y=VehUsage.NoClaimCount),
                      go.Bar(name='With Claims', x=VehUsage.VehUsage, y=VehUsage.ClaimCount)])
fig.update_layout(barmode='stack', title = 'Убыточность и безубыточность в зависимости от использования авто')
fig.show()

In [537]:
fig = go.Figure(data=[go.Bar(name='No Claims', x=df_group_Gender.Gender, y=df_group_Gender.NoClaimCount),
                      go.Bar(name='With Claims', x=df_group_Gender.Gender, y=df_group_Gender.ClaimCount)])
fig.update_layout(barmode='stack', title = 'Визуализация страхового случая по сравнению с нестраховым случаем в зависимости от пола')
fig.show()

In [538]:
px.bar(VehUsage, x = 'VehUsage', y = 'ClaimCount', title = 'Количество выплат в зависимости от профессии')

In [539]:
df_group_Gender.head()

,Gender,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim
0,Female,19073.537,43339,8.057192e+06,4087,39252,0.214276,1971.419679
1,Male,31808.614,71816,1.399185e+07,6782,65034,0.213213,2063.086503


In [540]:
px.bar(df_group_Gender, x = 'Gender', y = 'Freq', title = 'Частота страховых случаев в зависимости от пола')

In [541]:
px.bar(df_group_Gender, x = 'Gender', y = 'ClaimAmount')

In [542]:
df.iloc[:5, :]

,Exposure,LicAge,RecordBeg,RecordEnd,Gender,MariStat,SocioCateg,VehUsage,DrivAge,HasKmLimit,...,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,OutUseNb,RiskArea,PolicyCount,ClaimCount,NoClaimCount,DrivAge_5
145780,0.083,332,2004-01-01,2004-02-01,Male,Other,CSP50,Professional,46,0,...,1.0,0.0,0.0,0.0,0.0,9.0,1,False,1,9.0
145781,0.916,333,2004-02-01,NaN,Male,Other,CSP50,Professional,46,0,...,1.0,0.0,0.0,0.0,0.0,9.0,1,False,1,9.0
145782,0.550,173,2004-05-15,2004-12-03,Male,Other,CSP50,Private+trip to office,32,0,...,2.0,0.0,0.0,0.0,0.0,7.0,1,False,1,6.0
145783,0.089,364,2004-11-29,NaN,Female,Other,CSP55,Private+trip to office,52,0,...,0.0,0.0,0.0,0.0,0.0,8.0,1,False,1,10.0
145784,0.233,426,2004-02-07,2004-05-01,Male,Other,CSP60,Private,57,0,...,0.0,0.0,0.0,0.0,0.0,7.0,1,False,1,11.0


In [544]:
df_by_year = df

In [545]:
df_by_year

,Exposure,LicAge,RecordBeg,RecordEnd,Gender,MariStat,SocioCateg,VehUsage,DrivAge,HasKmLimit,...,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,OutUseNb,RiskArea,PolicyCount,ClaimCount,NoClaimCount,DrivAge_5
145780,0.083,332,2004-01-01,2004-02-01,Male,Other,CSP50,Professional,46,0,...,1.0,0.0,0.0,0.0,0.0,9.0,1,False,1,9.0
145781,0.916,333,2004-02-01,NaN,Male,Other,CSP50,Professional,46,0,...,1.0,0.0,0.0,0.0,0.0,9.0,1,False,1,9.0
145782,0.550,173,2004-05-15,2004-12-03,Male,Other,CSP50,Private+trip to office,32,0,...,2.0,0.0,0.0,0.0,0.0,7.0,1,False,1,6.0
145783,0.089,364,2004-11-29,NaN,Female,Other,CSP55,Private+trip to office,52,0,...,0.0,0.0,0.0,0.0,0.0,8.0,1,False,1,10.0
145784,0.233,426,2004-02-07,2004-05-01,Male,Other,CSP60,Private,57,0,...,0.0,0.0,0.0,0.0,0.0,7.0,1,False,1,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310975,0.423,238,2004-07-29,NaN,Male,Other,CSP50,Private+trip to office,39,0,...,0.0,0.0,0.0,0.0,4.0,8.0,1,False,1,8.0
310976,1.000,408,2004-01-01,NaN,Female,Other,CSP55,Private+trip to office,54,0,...,0.0,0.0,0.0,1.0,0.0,7.0,1,True,0,11.0
310977,0.805,211,2004-03-11,NaN,Male,Other,CSP55,Private+trip to office,35,0,...,0.0,1.0,0.0,0.0,0.0,7.0,1,False,1,7.0
310978,0.538,356,2004-01-01,2004-07-15,Male,Other,CSP50,Private,52,0,...,0.0,0.0,0.0,2.0,0.0,7.0,1,False,1,10.0


In [546]:
df_by_year['LicAgeYear'] = df_by_year.LicAge/365

df_by_year['LicAgeYear'].iloc[np.where((df_by_year['LicAgeYear']>0) & (df_by_year['LicAgeYear']<1))] = 0
df_by_year['LicAgeYear'].iloc[np.where((df_by_year['LicAgeYear']>1) & (df_by_year['LicAgeYear']<2))] = 1
df_by_year['LicAgeYear'].iloc[np.where((df_by_year['LicAgeYear']>2) & (df_by_year['LicAgeYear']<3))] = 2

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [548]:
df_by_year = df.groupby(df_by_year.LicAgeYear).sum()
df_by_year = df_by_year.reset_index()


In [549]:
df_by_year

,LicAgeYear,Exposure,LicAge,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,ClaimNbResp,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,OutUseNb,RiskArea,PolicyCount,ClaimCount,NoClaimCount,DrivAge_5
0,0.0,29053.926,14352487,2616637,5072,4299125,1.454879e+07,6770,16927.0,23515.0,4795.0,5736.0,26259.0,21613.0,528316.0,66984,6770,60214,523073.0
1,1.0,21574.038,22893260,2972546,7381,2496409,7.354507e+06,4059,12860.0,12770.0,4288.0,3132.0,15970.0,7774.0,373944.0,47638,4059,43579,594326.0
2,2.0,254.187,418565,44890,207,29054,1.457482e+05,40,210.0,93.0,58.0,19.0,91.0,56.0,4406.0,533,40,493,8994.0


In [550]:
px.bar(df_by_year, x = 'LicAgeYear', y = 'ClaimCount', title = 'Страховые случае в зависимости от возраста в годах')

In [552]:
fig = go.Figure(data=[go.Bar(name='No Claims', x=df_by_year.LicAgeYear, y=df_by_year.NoClaimCount),
                      go.Bar(name='With Claims', x=df_by_year.LicAgeYear, y=df_by_year.ClaimCount)])
fig.update_layout(barmode='stack', title = 'Визуализация страхового случая по сравнению с нестраховым случаем в зависимости от пола')
fig.show()

### Перевод в года не очень интересен, так как, видимо, датасет очень молодой.

In [555]:
df.head()

,Exposure,LicAge,RecordBeg,RecordEnd,Gender,MariStat,SocioCateg,VehUsage,DrivAge,HasKmLimit,...,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,OutUseNb,RiskArea,PolicyCount,ClaimCount,NoClaimCount,DrivAge_5,LicAgeYear
145780,0.083,332,2004-01-01,2004-02-01,Male,Other,CSP50,Professional,46,0,...,0.0,0.0,0.0,0.0,9.0,1,False,1,9.0,0.0
145781,0.916,333,2004-02-01,NaN,Male,Other,CSP50,Professional,46,0,...,0.0,0.0,0.0,0.0,9.0,1,False,1,9.0,0.0
145782,0.550,173,2004-05-15,2004-12-03,Male,Other,CSP50,Private+trip to office,32,0,...,0.0,0.0,0.0,0.0,7.0,1,False,1,6.0,0.0
145783,0.089,364,2004-11-29,NaN,Female,Other,CSP55,Private+trip to office,52,0,...,0.0,0.0,0.0,0.0,8.0,1,False,1,10.0,0.0
145784,0.233,426,2004-02-07,2004-05-01,Male,Other,CSP60,Private,57,0,...,0.0,0.0,0.0,0.0,7.0,1,False,1,11.0,1.0


In [ ]:
df_group_Gender_Age = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount', 'Gender', 'DrivAge']].groupby(['Gender', 'DrivAge']).sum()
df_group_Gender_Age = df_group_Gender_Age.reset_index()
df_group_Gender_Age['Freq'] = df_group_Gender_Age.ClaimCount / df_group_Gender_Age.Exposure
df_group_Gender_Age['AvgClaim'] = df_group_Gender_Age.ClaimAmount / df_group_Gender_Age.ClaimCount

In [572]:
df_group_SocioCateg_Gender = df[['Gender','SocioCateg','Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(['SocioCateg','Gender']).sum()
df_group_SocioCateg_Gender = df_group_SocioCateg_Gender.reset_index()
df_group_SocioCateg_Gender['Freq'] = df_group_SocioCateg_Gender.ClaimCount / df_group_SocioCateg_Gender.Exposure
df_group_SocioCateg_Gender['AvgClaim'] = df_group_SocioCateg_Gender.ClaimAmount / df_group_SocioCateg_Gender.ClaimCount

In [575]:
df_group_SocioCateg_Gender

,SocioCateg,Gender,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim
0,CSP1,Female,479.247,1041,206415.830211,109,932,0.227440,1893.723213
1,CSP1,Male,788.369,1696,436874.783384,181,1515,0.229588,2413.672836
2,CSP16,Male,0.023,1,0.000000,0,1,0.000000,NaN
3,CSP17,Male,0.173,1,0.000000,0,1,0.000000,NaN
4,CSP19,Male,0.042,1,0.000000,0,1,0.000000,NaN
...,...,...,...,...,...,...,...,...,...
68,CSP66,Male,670.753,1433,310803.253776,172,1261,0.256428,1806.995661
69,CSP7,Female,1.998,4,-822.829305,0,4,0.000000,-inf
70,CSP7,Male,4.120,8,386.158912,1,7,0.242718,386.158912
71,CSP70,Female,0.071,1,0.000000,0,1,0.000000,NaN


In [586]:
fig = px.bar(df_group_SocioCateg_Gender, x='SocioCateg', y='Freq', color='Gender', title='Зависимость частоты убытков от пола и социального статуса')
fig.show()

Среди мужчин в социальном статусе 62 есть какой то всплеск по частоте страховых случаев.

In [624]:
df.loc[df['SocioCateg'] == 'CSP62']

,Exposure,LicAge,RecordBeg,RecordEnd,Gender,MariStat,SocioCateg,VehUsage,DrivAge,HasKmLimit,...,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,OutUseNb,RiskArea,PolicyCount,ClaimCount,NoClaimCount,DrivAge_5,LicAgeYear
258104,0.122,402,2004-11-17,NaN,Male,Other,CSP62,Professional,51,0,...,0.0,0.0,1.0,0.0,11.0,1,False,1,10.0,1.0
275366,0.036,344,2004-11-27,2004-12-10,Male,Other,CSP62,Professional,48,0,...,0.0,0.0,0.0,0.0,6.0,1,True,0,10.0,0.0
283304,0.176,229,2004-10-28,NaN,Male,Alone,CSP62,Professional,37,0,...,0.0,1.0,1.0,3.0,9.0,1,False,1,7.0,0.0


### В этой группе получается только 3 клиента мужчины и один страховой случай. Таким образом из-за малой выборки вспелск страховых случаев можно считать недостоверным. Исключим посмотрим снова.

In [643]:
df_exept_CSP62 = df[['Gender','SocioCateg','Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(['SocioCateg','Gender']).sum()
df_exept_CSP62 = df_exept_CSP62.reset_index()
df_exept_CSP62['Freq'] = df_exept_CSP62.ClaimCount / df_exept_CSP62.Exposure
df_exept_CSP62['AvgClaim'] = df_exept_CSP62.ClaimAmount / df_exept_CSP62.ClaimCount
df_exept_CSP62 = df_exept_CSP62.drop(np.where(df_exept_CSP62['SocioCateg']== 'CSP62')[0])


In [645]:
fig = px.bar(df_exept_CSP62, x='SocioCateg', y='Freq', color='Gender', title='Зависимость частоты убытков от пола и социального статуса')
fig.show()